In [20]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/Data/Cleaned_Null_Values+Sale_Data.csv')
df

,Unnamed: 0,status,price,bed,bath,acre_lot,city,state,zip_code,house_size,months_since_prev_sold
0,0,for_sale,105000.0,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.000000,0
1,1,for_sale,80000.0,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.000000,0
2,2,for_sale,67000.0,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.000000,0
3,3,for_sale,145000.0,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.000000,0
4,4,for_sale,65000.0,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,2714.471335,0
...,...,...,...,...,...,...,...,...,...,...,...
1411580,1414368,for_sale,299000.0,3.0,2.0,0.67,Ketchikan,Alaska,99901.0,2714.471335,0
1411581,1414369,for_sale,299000.0,3.0,2.0,0.67,Ketchikan,Alaska,99901.0,2714.471335,0
1411582,1414370,for_sale,199000.0,3.0,2.0,1.31,Ketchikan,Alaska,99901.0,2714.471335,0
1411583,1414371,for_sale,1575000.0,3.0,2.0,292.58,Ketchikan,Alaska,99901.0,2714.471335,0


## Feature Engineering

In [22]:
"""
Cell generated by Data Wrangler.
"""
import pickle
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def clean_data(df):
    os.makedirs('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/pipelines',exist_ok=True)
    le = LabelEncoder()    
    one = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    status_encoded = one.fit_transform(df[['status']])
    with open('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/pipelines/one_hot_encoding.pkl','wb') as f:
        pickle.dump(one, f)  # save encoder, not data
    status_encoded_df = pd.DataFrame(status_encoded,columns=one.get_feature_names_out(['status']))
    df = pd.concat([df.drop(['status'],axis=1),status_encoded_df],axis=1) 
    df['city'] = le.fit_transform(df['city'] + ", " + df['state'])
    with open('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/pipelines/label_encoding.pkl','wb') as f:
        pickle.dump(le, f)
    df = df.drop(['state'],axis=1)
    df['acre_lot'] = df['acre_lot'] * 43560
    df['total_house_size'] = df['acre_lot'] + df['house_size']
    df['total_rooms'] = df['bed'] + df['bath']
    df['price_per_sqft'] = df['total_house_size'] / df['price']
    df = df.drop(columns=df.columns[df.isin([np.inf, -np.inf]).any()])
    df = df.rename(columns={'price':'cost',
                            'bed':'bedrooms',
                            'bath':'bathrooms',
                            'acre_lot':'sqft_lot',
                            'house_size':'house_lot',
                            'city':'location',
                            'months_since_prev_sold':'last_sold(months)'})
    # Round columns 'cost', 'sqft_lot' and 3 other columns (Number of decimals: 4)
    df = df.round({'cost': 4, 'sqft_lot': 4, 'house_lot': 4, 'total_house_size': 4, 'price_per_sqft': 4})
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,Unnamed: 0,cost,bedrooms,bathrooms,sqft_lot,location,zip_code,house_lot,last_sold(months),status_for_sale,status_ready_to_build,total_house_size,total_rooms
0,0,105000.0,3.0,2.0,5227.2,137,601.0,920.0000,0,1.0,0.0,6147.2000,5.0
1,1,80000.0,4.0,2.0,3484.8,137,601.0,1527.0000,0,1.0,0.0,5011.8000,6.0
2,2,67000.0,2.0,1.0,6534.0,13201,795.0,748.0000,0,1.0,0.0,7282.0000,3.0
3,3,145000.0,4.0,2.0,4356.0,21140,731.0,1800.0000,0,1.0,0.0,6156.0000,6.0
4,4,65000.0,6.0,2.0,2178.0,16317,680.0,2714.4713,0,1.0,0.0,4892.4713,8.0


In [23]:
df = df_clean.copy()

In [24]:
df

,Unnamed: 0,cost,bedrooms,bathrooms,sqft_lot,location,zip_code,house_lot,last_sold(months),status_for_sale,status_ready_to_build,total_house_size,total_rooms
0,0,105000.0,3.0,2.0,5227.2,137,601.0,920.0000,0,1.0,0.0,6.147200e+03,5.0
1,1,80000.0,4.0,2.0,3484.8,137,601.0,1527.0000,0,1.0,0.0,5.011800e+03,6.0
2,2,67000.0,2.0,1.0,6534.0,13201,795.0,748.0000,0,1.0,0.0,7.282000e+03,3.0
3,3,145000.0,4.0,2.0,4356.0,21140,731.0,1800.0000,0,1.0,0.0,6.156000e+03,6.0
4,4,65000.0,6.0,2.0,2178.0,16317,680.0,2714.4713,0,1.0,0.0,4.892471e+03,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411580,1414368,299000.0,3.0,2.0,29185.2,13490,99901.0,2714.4713,0,1.0,0.0,3.189967e+04,5.0
1411581,1414369,299000.0,3.0,2.0,29185.2,13490,99901.0,2714.4713,0,1.0,0.0,3.189967e+04,5.0
1411582,1414370,199000.0,3.0,2.0,57063.6,13490,99901.0,2714.4713,0,1.0,0.0,5.977807e+04,5.0
1411583,1414371,1575000.0,3.0,2.0,12744784.8,13490,99901.0,2714.4713,0,1.0,0.0,1.274750e+07,5.0


In [6]:
df.isna().sum()

Unnamed: 0           0
cost                 0
bedrooms             0
bathrooms            0
sqft_lot             0
location             0
zip_code             0
house_lot            0
last_sold(months)    0
ready_to_build       0
total_house_size     0
total_rooms          0
dtype: int64

In [7]:
df.to_csv('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/Data/Cleaned_Null_Values+Sale_Data+Feature_Engineering.csv')